# ENGIE QTEM Data Challenge – 2025  

This notebook is a comprehensive and reproducible workflow to solve the ENGIE QTEM Data Challenge. The challenge is focused on optimizing a renewable energy asset portfolio based on weather, production, and market pricing data.

### Key Business Questions:
- Which combination of renewable energy assets yields the highest production with the least variability?
- How do revenue considerations (market price data) affect the optimal portfolio selection?
- What are the differences between production-driven and revenue-driven optimization?
- How would the results change if real-world inefficiencies (e.g. curtailments) were removed?

---

### Final Deliverables:
- Reproducible optimization models (production & revenue)
- Portfolio performance metrics (mean output, standard deviation, volatility)
- Comparative analysis & sensitivity insights
- Final report and presentation-ready visualizations

---

### Data Overview:
- **Solar Production Sites:** 71 (Belgium, Germany, Netherlands)
- **Wind Sites:** 99 (onshore + offshore)
- **Weather Variables:** Temperature, wind speed/direction, cloud cover, solar radiation
- **Prices:** Quarter-hourly and hourly market prices (Day-Ahead, Intraday, ISP)
- **Metadata:** Site locations, variable definitions

---

## Available Data:

- **Solar Data Folder:** Contains files for each solar site:
  - Dew Point (`d2m`)
  - Total Cloud Cover (`tcc`)
  - Temperature (`t2m`)
  - Solar Radiation (`ssr`)
  - Wind Angle at 10m (`angle10`)
  - Wind Speed at 10m (`speed10`)
  - Load Factor (`factor`)

- **Wind Data Folder:** Contains files for each wind site:
  - Wind Angle at 100m (`angle100`)
  - Wind Speed at 100m (`speed100`)
  - Load Factor (`factor`)

- **Price and Liquidity Data:**
  - Day Ahead and Intraday prices.

- **Additional Files:**
  - Sites anonymized data (`sites_anonymized.csv`)
  - Data Dictionary (`data_dictionary.xlsx`)

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from glob import glob

# 1. Comprehensive Data Assessment

In this section, we:
- Explore metadata to understand the site distribution
- Examine weather, production, and price data structure
- Identify missing values, timestamp irregularities, and data format issues
- Prepare for integration and cleaning

In [33]:
# Load metadata
sites_df = pd.read_csv("/Users/hossameldinelhendawy/Documents/QDC-Lib/sites_anonymized.csv")
price_df = pd.read_csv("/Users/hossameldinelhendawy/Documents/QDC-Lib/intraday_indices_prices_and_liquidity.csv")

# Datetime column modification
price_df.rename(columns={price_df.columns[0]: "datetime"}, inplace=True)
price_df['datetime'] = pd.to_datetime(price_df['datetime'])

/var/folders/5f/8x4b66f12jxblmgjlz1qymv80000gn/T/ipykernel_12846/3748134077.py:7: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  price_df['datetime'] = pd.to_datetime(price_df['datetime'])


In [34]:
# Selection of relevant price columns for Parts 1–4
columns_to_keep = ['datetime']

# Add DA prices
for country in ['BE', 'DE', 'NL']:
    col = f'DA_{country}'
    if col in price_df.columns:
        columns_to_keep.append(col)

# Add hourly and QH intraday prices
intraday_types = ['ID1', 'ID3', 'IDFull']
resolutions = ['Hourly', 'QH']

for market in intraday_types:
    for resolution in resolutions:
        for country in ['BE', 'DE', 'NL']:
            col = f'{market}_{resolution}_{country}_price'
            if col in price_df.columns:
                columns_to_keep.append(col)

# Add Imbalance prices
for country in ['BE', 'DE', 'NL']:
    # Flat ISP
    col_flat = f'ISP_{country}'
    if col_flat in price_df.columns:
        columns_to_keep.append(col_flat)
    # NL-specific imbalance split
    col_short = f'ISP_SHORT_{country}'
    col_long = f'ISP_LONG_{country}'
    if col_short in price_df.columns:
        columns_to_keep.append(col_short)
    if col_long in price_df.columns:
        columns_to_keep.append(col_long)

# Apply filter
price_cleaned_df = price_df[columns_to_keep].copy()

### 📊 Price & Liquidity Data Cleaning Summary

We started with a raw pricing dataset containing 59 columns and 132,310 timestamped records, covering multiple market types (Day-Ahead, Intraday, Imbalance) across Belgium (BE), Germany (DE), the Netherlands (NL), and France (FR).

#### ✅ Actions Taken:
- Removed all `volume` columns, which are not required in the ENGIE optimization objective (no use for liquidity or transaction volume).
- Dropped all columns related to France (`FR`) since the challenge scope is limited to BE, DE, and NL.
- Excluded rarely populated or redundant formats such as half-hour (HH) pricing and kept only **Hourly and Quarter-Hourly (QH)** series where reasonably populated.
- Retained all Day-Ahead (`DA_*`) pricing columns.
- Retained relevant Intraday pricing layers: `ID1`, `ID3`, and `IDFull`, for both `Hourly` and `QH` formats.
- Retained flat imbalance prices `ISP_*`, as well as `ISP_SHORT_NL` and `ISP_LONG_NL` for modeling imbalance market logic in Part 2 or Part 4.

#### 📦 Resulting Dataset:
- Final shape: **132,310 rows × 26 columns**
- Includes all pricing columns required for Parts **2, 3, and 4** of the challenge.
- Structured to support reproducible modeling and easy merging with production and weather data.